# S1. Red convolucional para MNIST



Como en la anterior práctica, primero importamos el conjunto de MNIST y lo normalizamos, pero sin convertir las imágenes en vectores unidimensionales, ya que vamos a trabajar con redes convolucionales que explotan la estructura 2D de las imágenes.

In [2]:
## Importar y normalizar datos

from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print('training set', x_train.shape)
print('val set', x_val.shape)

training set (60000, 28, 28)
test set (10000, 28, 28)
training set (48000, 28, 28)
val set (12000, 28, 28)


## Modelo base
 Partiremos de una topología base que toma la red MLP de la última sesión de la primera práctica (dos capas densas de 1024 neuronas), y le incorpora un par de capas convolucionales cada una seguida por average pooling inspirada en la arquitectura LeNet (1998) propuesta por Yann LeCun para MNIST. 
 

In [4]:
from keras import Sequential
from keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

model = Sequential()

model.add(Input((28,28,1)))
model.add(Conv2D(filters=6, kernel_size=(5,5), activation='relu', input_shape=(28,28,1)))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu'))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(Dense(units=1024, activation='relu'))
model.add(Dense(units=10, activation = 'softmax'))

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=25
batch_size=128
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])  

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Epoch 1/25
375/375 [==============================] - ETA: 0s - loss: 0.2707 - accuracy: 0.9205
Epoch 1: val_accuracy improved from -inf to 0.96925, saving model to best_model.h5
375/375 [==============================] - 3s 5ms/step - loss: 0.2707 - accuracy: 0.9205 - val_loss: 0.1027 - val_accuracy: 0.9693 - lr: 0.0010
Epoch 2/25
375/375 [==============================] - ETA: 0s - loss: 0.0786 - accuracy: 0.9758
Epoch 2: val_accuracy improved from 0.96925 to 0.97875, saving model to best_model.h5
375/375 [==============================] - 2s 5ms/step - loss: 0.0786 - accuracy: 0.9758 - val_loss: 0.0689 - val_accuracy: 0.9787 - lr: 0.0010
Epoch 3/25
364/375 [============================>.] - ETA: 0s - loss: 0.0536 - accuracy: 0.9842
Epoch 3: val_accuracy improved from 0.97875 to 0.98475, saving model to best_model.h5
375/375 [==============================] - 2s 6ms/step - loss: 0.0533 - accuracy: 0.9842 - val_loss: 0.0532 - val_accuracy: 0.9847 - lr: 0.0010
Epoch 4/25
365/375 [=====

## Ejercicio:

Probar las técnicas presentadas en la práctica 1 (dropout, batchnorm y aumento de datos) para obtener un acierto en test > 99%, incluso mejor que la obtenida con redes MLP.

## Solución:

## Regularización l2 (o l1)

La regularización l2 consiste en añadir a la función de coste una penalización proporcional a la norma l2 de los pesos del modelo. De esta forma, se penaliza a los pesos que tengan un valor alto, forzando a que los pesos tengan valores pequeños. Esto se conoce como regularización l2. También podríamos hacer lo mismo con regularización l1 o con ambas (lo que se conoce como *Elastic net*)


In [5]:
## Teniendo en cuenta el modelo base añade regularización L2 a las capas densas
from keras import Sequential
from keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.regularizers import l2

model = Sequential()

model.add(Input((28,28,1)))
model.add(Conv2D(filters=6, kernel_size=(5,5), activation='relu', input_shape=(28,28,1)))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu'))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Flatten())
model.add(Dense(units=1024, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dense(units=1024, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dense(units=10, activation = 'softmax'))

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=25
batch_size=128
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])  

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Epoch 1/25
367/375 [============================>.] - ETA: 0s - loss: 1.4852 - accuracy: 0.8688
Epoch 1: val_accuracy improved from -inf to 0.93367, saving model to best_model.h5
375/375 [==============================] - 2s 4ms/step - loss: 1.4626 - accuracy: 0.8700 - val_loss: 0.4141 - val_accuracy: 0.9337 - lr: 0.0010
Epoch 2/25
357/375 [===========================>..] - ETA: 0s - loss: 0.3290 - accuracy: 0.9476
Epoch 2: val_accuracy improved from 0.93367 to 0.95400, saving model to best_model.h5
375/375 [==============================] - 2s 5ms/step - loss: 0.3269 - accuracy: 0.9477 - val_loss: 0.2779 - val_accuracy: 0.9540 - lr: 0.0010
Epoch 3/25
369/375 [============================>.] - ETA: 0s - loss: 0.2538 - accuracy: 0.9583
Epoch 3: val_accuracy improved from 0.95400 to 0.96458, saving model to best_model.h5
375/375 [==============================] - 2s 5ms/step - loss: 0.2534 - accuracy: 0.9585 - val_loss: 0.2249 - val_accuracy: 0.9646 - lr: 0.0010
Epoch 4/25
369/375 [=====

## Dropout

El dropout es una técnica de regularización que consiste en eliminar aleatoriamente un porcentaje de las neuronas de la red durante el entrenamiento. De esta forma, se evita que la red se sobreajuste a los datos de entrenamiento y se mejora la generalización del modelo.


In [19]:
## Teniendo en cuenta el modelo base añade regularización de tipo dropout a las capas densas
from keras import Sequential
from keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

model = Sequential()

model.add(Input((28,28,1)))
model.add(Conv2D(filters=6, kernel_size=(5,5), activation='relu', input_shape=(28,28,1)))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu'))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=10, activation = 'softmax'))

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=25
batch_size=128
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])  

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')


Epoch 1/25
365/375 [============================>.] - ETA: 0s - loss: 0.3298 - accuracy: 0.8968
Epoch 1: val_accuracy improved from -inf to 0.96983, saving model to best_model.h5
375/375 [==============================] - 2s 5ms/step - loss: 0.3254 - accuracy: 0.8983 - val_loss: 0.1076 - val_accuracy: 0.9698 - lr: 0.0100
Epoch 2/25
 53/375 [===>..........................] - ETA: 0s - loss: 0.1666 - accuracy: 0.9562

/home/jorcisai/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


374/375 [============================>.] - ETA: 0s - loss: 0.1623 - accuracy: 0.9571
Epoch 2: val_accuracy improved from 0.96983 to 0.97633, saving model to best_model.h5
375/375 [==============================] - 2s 4ms/step - loss: 0.1620 - accuracy: 0.9571 - val_loss: 0.0990 - val_accuracy: 0.9763 - lr: 0.0100
Epoch 3/25
371/375 [============================>.] - ETA: 0s - loss: 0.1432 - accuracy: 0.9632
Epoch 3: val_accuracy did not improve from 0.97633
375/375 [==============================] - 2s 5ms/step - loss: 0.1431 - accuracy: 0.9633 - val_loss: 0.0989 - val_accuracy: 0.9756 - lr: 0.0100
Epoch 4/25
372/375 [============================>.] - ETA: 0s - loss: 0.1420 - accuracy: 0.9648
Epoch 4: val_accuracy improved from 0.97633 to 0.97950, saving model to best_model.h5
375/375 [==============================] - 2s 5ms/step - loss: 0.1420 - accuracy: 0.9647 - val_loss: 0.0819 - val_accuracy: 0.9795 - lr: 0.0100
Epoch 5/25
365/375 [============================>.] - ETA: 0s - loss

## Normalización BatchNorm

La normalización BatchNorm consiste en normalizar la salida de una capa de la red neuronal para que tenga media 0 y varianza 1. De esta forma, se consigue que la red neuronal pueda entrenarse más rápido y que sea más robusta a cambios en los pesos de las capas anteriores.


In [20]:
## Teniendo en cuenta el modelo base añade normalización BatchNorm
from keras import Sequential
from keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

model = Sequential()

model.add(Input((28,28,1)))
model.add(Conv2D(filters=6, kernel_size=(5,5), activation='relu', input_shape=(28,28,1)))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu'))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=10, activation = 'softmax'))

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=25
batch_size=128
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])  

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Epoch 1/25
372/375 [============================>.] - ETA: 0s - loss: 0.2330 - accuracy: 0.9422
Epoch 1: val_accuracy improved from -inf to 0.97050, saving model to best_model.h5
375/375 [==============================] - 3s 6ms/step - loss: 0.2325 - accuracy: 0.9424 - val_loss: 0.1222 - val_accuracy: 0.9705 - lr: 0.0100
Epoch 2/25
368/375 [============================>.] - ETA: 0s - loss: 0.0762 - accuracy: 0.9765
Epoch 2: val_accuracy improved from 0.97050 to 0.97633, saving model to best_model.h5
375/375 [==============================] - 3s 7ms/step - loss: 0.0764 - accuracy: 0.9765 - val_loss: 0.0874 - val_accuracy: 0.9763 - lr: 0.0100
Epoch 3/25
372/375 [============================>.] - ETA: 0s - loss: 0.0560 - accuracy: 0.9831
Epoch 3: val_accuracy improved from 0.97633 to 0.98158, saving model to best_model.h5
375/375 [==============================] - 3s 7ms/step - loss: 0.0558 - accuracy: 0.9831 - val_loss: 0.0691 - val_accuracy: 0.9816 - lr: 0.0100
Epoch 4/25
369/375 [=====

## Aumentado de datos

El aumentado de datos consiste en generar nuevos datos de entrenamiento a partir de los datos de entrenamiento originales. De esta forma, se consigue que el modelo sea más robusto y que se generalice mejor a datos que no ha visto durante el entrenamiento.

En nuestro caso para los dígitos de la MNIST vamos a realizar un aumento de datos de la siguiente forma:

- Rotación aleatoria de la imagen entre -30 y 30 grados.
- Traslación aleatoria de la imagen entre -3 y 3 píxeles en horizontal y vertical.
- Escalado aleatorio de la imagen entre 0.8 y 1.2.
- Inversión aleatoria de la imagen en horizontal y vertical. **NO!!!**

El aumentado de datos se ejecuta en CPU y ralentiza el entrenamiento.

Normalmente además, se necesitarán más epochs para entrenar el modelo. 

In [22]:
## Implementamos en el ejemplo base el aumentado de datos
from keras import Sequential
from keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')

## Importante: ImageDataGenerator espera una imagen con 3 canales, necesitamos hacer reshape
x_train = x_train.reshape(48000, 28, 28, 1)
x_val = x_val.reshape(12000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

## Ajustamos el generador de datos
datagen.fit(x_train)

model = Sequential()

model.add(Input((28,28,1)))
model.add(Conv2D(filters=6, kernel_size=(5,5), activation='relu', input_shape=(28,28,1)))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu'))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=10, activation = 'softmax'))

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


epochs=25
batch_size=128
## Entrenamos con el generador de datos en lugar de con el dataset
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')



Epoch 1/25
372/375 [============================>.] - ETA: 0s - loss: 0.5746 - accuracy: 0.8426
Epoch 1: val_accuracy improved from -inf to 0.95550, saving model to best_model.h5
375/375 [==============================] - 7s 16ms/step - loss: 0.5723 - accuracy: 0.8433 - val_loss: 0.1538 - val_accuracy: 0.9555 - lr: 0.0100
Epoch 2/25
375/375 [==============================] - ETA: 0s - loss: 0.2089 - accuracy: 0.9358
Epoch 2: val_accuracy improved from 0.95550 to 0.97275, saving model to best_model.h5
375/375 [==============================] - 7s 18ms/step - loss: 0.2089 - accuracy: 0.9358 - val_loss: 0.0865 - val_accuracy: 0.9728 - lr: 0.0100
Epoch 3/25
372/375 [============================>.] - ETA: 0s - loss: 0.1593 - accuracy: 0.9518
Epoch 3: val_accuracy improved from 0.97275 to 0.97750, saving model to best_model.h5
375/375 [==============================] - 6s 17ms/step - loss: 0.1593 - accuracy: 0.9518 - val_loss: 0.0724 - val_accuracy: 0.9775 - lr: 0.0100
Epoch 4/25
375/375 [==